In [16]:
import torch
import torch.nn as nn
import torch.optim as optim

In [17]:
data  = "Implement RNN using Pytorch"
chars = sorted(list(set(data)))
vocab_size = len(chars)
char_to_ix = {ch: i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}

input_indices = [char_to_ix[ch] for ch in data[:-1]]
target_indices = [char_to_ix[ch] for ch in data[1:]]


In [18]:
class PytorchRNN(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(PytorchRNN,self).__init__()
    self.hidden_size = hidden_size

    #input_size is vocab_size (one-hot encoding)
    self.rnn = nn.RNN(input_size,hidden_size,num_layers=1,batch_first = True)

    #Output layer (maps hidden state to vocab space)
    self.fc = nn.Linear(hidden_size,output_size)

  def forward(self,input_tensor,hidden_state):
    rnn_out , hidden_state_out = self.rnn(input_tensor,hidden_state)

    output = self.fc(rnn_out)

    return output,hidden_state_out


In [19]:
hidden_size = 100
model = PytorchRNN(vocab_size,hidden_size,vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.01)

X = torch.zeros(1,len(input_indices),vocab_size)
for t,idx in enumerate(input_indices):
  X[0,t,idx] = 1.0
Y = torch.LongTensor(target_indices).unsqueeze(0)


In [20]:
for i in range(1000):
  h0 = torch.zeros(1,1,hidden_size)
  optimizer.zero_grad()
  output,h_final = model(X,h0)

  loss = criterion(output.view(-1,vocab_size),Y.view(-1))

  loss.backward()
  optimizer.step()

  if i%100 == 0:
    print(f"Iter {i} , Loss: {loss.item():.4f}")



Iter 0 , Loss: 2.9893
Iter 100 , Loss: 0.0004
Iter 200 , Loss: 0.0002
Iter 300 , Loss: 0.0002
Iter 400 , Loss: 0.0001
Iter 500 , Loss: 0.0001
Iter 600 , Loss: 0.0001
Iter 700 , Loss: 0.0001
Iter 800 , Loss: 0.0000
Iter 900 , Loss: 0.0000


In [21]:
def sample_pytorch(model,start_char,num_chars):
  model.eval()
  h = torch.zeros(1,1,model.hidden_size)

  input_idx = char_to_ix[start_char]
  input_one_hot = torch.zeros(1,1,vocab_size)
  input_one_hot[0,0,input_idx] = 1.0

  text = start_char

  with torch.no_grad():
    for t in range(num_chars):
      output, h = model(input_one_hot,h)

      p = torch.softmax(output.squeeze(0).squeeze(0), dim = 0)

      ix = torch.multinomial(p,1).item()

      text += ix_to_char[ix]

      input_one_hot.zero_()
      input_one_hot[0,0,ix]  = 1.0

  return text

print(f"Start char: {data[0]}")
print(f"Generated text : {sample_pytorch(model,data[0],len(data) -1 )}")

Start char: I
Generated text : Implement RNN using Pytorch
